In [2]:
import glob
import sncosmo
import os
from astropy.table import Table
import pandas as pd
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from read_yse_ztf_snana_dir import read_YSE_ZTF_snana_dir
import light_curve
from light_curve import VillarFit
import extinction
from extinction import fm07, apply, remove
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import warnings
import glob
import time
import datetime
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import csv
from decimal import Decimal, ROUND_HALF_UP

python(51945) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


# Setting up Sims in the same format as Real Data

In [3]:
def read_snana_fits(dir_path):
    snid_list = []
    meta_list = []
    yse_ztf_fp_df_list = []
    meta = {}
    objid = ""
    if (dir_path == "../Retrain_RAPID/for_YSE/data/PALEO_YSE_MODEL43_CCSNe_V19_yselc_v2_52LCs"):
        objid = "yselc_v2"
    elif (dir_path == "../../../yse_ztf_sims/PALEO_YSE_ZTF_MODEL01"): #WILL NEED TO UPDATE FOR LOCAL MACHINE
        objid = "SNIa" #Change if you need other version 
    elif (dir_path == "/Users/sammysharief/Downloads/YSE_DR1_SIMS_60k_SNR4_grizXY_SIMLIB_FLUXCALERR_COR_220427/PALEO_YSE_ZTF_MODEL37"):
        objid = "SNIbc"
    elif (dir_path == "/Users/sammysharief/Downloads/YSE_DR1_SIMS_60k_SNR4_grizXY_SIMLIB_FLUXCALERR_COR_220427/PALEO_YSE_ZTF_MODEL20"):
        objid = "SNII"
        
    heads = sorted(glob.glob(os.path.join(dir_path, '*_HEAD.FITS.gz')))
    phots = sorted(glob.glob(os.path.join(dir_path, '*_PHOT.FITS.gz')))
    assert len(heads) != 0, 'no *_HEAD_FITS.gz are found'
    assert len(heads) == len(phots), 'there are different number of HEAD and PHOT files'
    
    for head, phot in zip(heads, phots): #lots of LCs per head, phot files, so do a few to start
        i = head.find('_HEAD.FITS.gz')
        assert head[:i] == phot[:i], f'HEAD and PHOT files name mismatch: {head}, {phot}'
        filename = head[:i].split('/')[-1].split('.')[0]
        head_title = head[88:92]         
        for LCnum, lc in enumerate(sncosmo.read_snana_fits(head, phot)): # remember: multiple SN in single HEAD/PHOT file
            lc.meta['redshift'] = lc.meta['SIM_REDSHIFT_HOST']
            redshift = lc.meta['redshift']
            checkSnid = lc.meta['SNID']
            checkSnid = checkSnid.decode('UTF-8')
            redshift = np.round(redshift, 2)
            ra = lc.meta['RA']
            dec = lc.meta['DEC']
            
            lc.keep_columns(['MJD', 'BAND', 'FLUXCAL', 'FLUXCALERR', 'PHOTFLAG'])
            
            totalResidual = 0
            fitNew = []
            # Astropy Table --> Pandas df
            df = lc.to_pandas()
            df = df.rename(columns={
                "BAND": "PASSBAND", 
                "FLUXCAL": "FLUX",
                "FLUXCALERR": "FLUXERR", 
                "PHOTFLAG": "PHOTFLAG"
            })
            
            df['PASSBAND']= df['PASSBAND'].str.decode("utf-8") # turn Bytes into str
            snr = []
            magerr = []
            
            snr = df['FLUX']/df['FLUXERR'] 
            df['MAG'] = np.array(-2.5*np.log10(np.abs(df['FLUX'])))+27.5
            magerr = 1.086/snr
            df['MAGERR'] = magerr
            #Made DF and have all the info
            meta['object_id'] = objid
            meta['RA'] = ra
            meta['DEC'] = dec
            meta['redshift'] = redshift
            meta['SNID'] = checkSnid
            meta['Head'] = head_title
            #Add data to respective data structure
            snid_list.append(objid)
            meta_copy = meta.copy()
            meta_list.append(meta_copy)
            yse_ztf_fp_df_list.append(df)
    return snid_list, meta_list, yse_ztf_fp_df_list

# Plotting YSE DR1 Object and Closest Simulated Object

In [4]:
#SPLIT UP READING AND PLOTTING
#Photflag --> remove rows that photflag == 0 
def plotData(snid_list, meta_list, yse_ztf_fp_df_list, plotType, thresehold):
    mcmc_niter=(4096)
    lmsder_niter=64
    fit = VillarFit('mcmc-lmsder', mcmc_niter=mcmc_niter, lmsder_niter=lmsder_niter) #increase mcmc_niter if fits are not good
    print(plotType)
    print(thresehold)
    df = pd.DataFrame()   
    mebvExist = 0
    num_of_bands = 0
    no_points_to_plot_mag = 0
    no_points_to_plot_flux = 0
    title = ""
    if (meta_list[0]['object_id'][0] == '2'):
        title = f'Real_Data_{plotType}'
        mebvExist = 1
    else:
        title =  f'{snid_list[0]}_{plotType}_sims'

    #print("title: ", title)
    with PdfPages(f'{title}.pdf') as pdf:
        for ix, df in enumerate(yse_ztf_fp_df_list):
            if (mebvExist == 1):
                objid = snid_list[ix]
            else:
                head = meta_list[ix]['Head']
                uniqueObjId = meta_list[ix]['SNID']
                objid = f'{head}_{uniqueObjId}_SNIa' #Update if not SN Ia
                #print(objid)
            df = df.reset_index(drop=True)
            
            df['SNR'] = df['FLUX']/df['FLUXERR'] 
            redshift = meta_list[ix]['redshift']
            if (redshift < 0):
                redshift = "N/A"
            peakMJD = df.iloc[df['FLUX'].idxmax()]["MJD"] 
            for i in df['MJD']:
                if ( np.abs(peakMJD - i) > 200):
                    df.drop(df.index[df['MJD'] == i], inplace=True)
            if (mebvExist == 1):
                D_id_color = {"X": u"olivedrab",
                  "Y": u"darkred",
                  "g": u"g", 
                  "r": u"r",
                  "i": u"m",
                  "z": u"k"}
            else:
                D_id_color = {
                      "X ": u"olivedrab",
                      "Y ": u"darkred",
                      "g ": u"g", 
                      "r ": u"r",
                      "i ": u"m",
                      "z ": u"k"}

            df['PLOTCOLOR'] = df.PASSBAND.map(D_id_color)
            results_pb, models_pb = [], []
            totalResidual = 0
            passband = df['PASSBAND']
            count = 0
            countPlot = 0
            noFitDf = df.copy()
            if (plotType == "MAG"):
                plt.gca().invert_yaxis()
            if (ix == -10): #change ix if you need to debug and don't wanna take forever
                break
            for pb in np.unique(passband):
                mask = (df['PASSBAND'] == pb) & (df['PHOTFLAG'] != 0) & (df['SNR'] > 4) & (df['FLUX'] > 0)#no non-detections
                df_pb = df[mask] 
                df_pb = df_pb.sort_values(by=['MJD'], ascending=True) # w/ photpipe, some have (near) duplicate MJDs separated
                df_pb = df_pb.drop_duplicates(subset=['MJD']) # Remove all duplicate MJD rows (prob w/ photpipe output)    
                mjd_pb = np.array(df_pb['MJD'])
                mag_pb = np.array(df_pb['MAG'])
                magerr_pb = np.array(df_pb['MAGERR'])
                flux_pb = np.array(df_pb['FLUX'])
                fluxerr_pb = np.array(df_pb['FLUXERR'])
                passband_pb = np.array(df_pb['PASSBAND'])
                photflag_pb = np.array(df_pb['PHOTFLAG'])
                plotcolor_pb = df_pb['PLOTCOLOR'] 
                if (len(mjd_pb) < 8):
                    df_pb.drop(df_pb.index[df_pb['PASSBAND'] == pb], inplace=True)
                    count = count + 1
                    continue
                else:
                    mjd_pb = np.float64(mjd_pb)
                    flux_pb = np.float64(flux_pb)
                    fluxerr_pb = np.float64(fluxerr_pb)
                    result_pb = fit(mjd_pb, flux_pb, fluxerr_pb, sorted=True) #sorted=True now b/c we use sort_value
                    results_pb.append(result_pb)
                    max_mjd_pb = np.max(mjd_pb)
                    min_mjd_pb = np.min(mjd_pb)
                    day0p1 = (max_mjd_pb-min_mjd_pb)/0.1
                    mjd_grid_pb = np.linspace(min_mjd_pb, max_mjd_pb, int(day0p1))
                    # Minimum number of observations: **8** per band to plot
                    model_pb = VillarFit.model(mjd_grid_pb, result_pb)
                    models_pb.append(model_pb)
                    #Villar fit on actual mjd data so you have the same amount of data points
                    model_pbNew = VillarFit.model(mjd_pb, result_pb) #Just using actual data --> It gives it to you in flux
                    model_pbMagNew = np.array(-2.5*np.log10(np.abs(model_pbNew)))+27.5 #Converting to Mag
                    #Residuals = np.abs(mag - model_pbMagNew)
                    Residuals_pb = np.abs(mag_pb - model_pbMagNew)
                    #Residuals = np.abs((flux - model_pbNew)/fluxerr)
                    df_pb['Residuals'] = np.array(Residuals_pb)
                    if not os.path.exists(f'../Photpipe/yse_ztf_fp_VillarFit_noNonDetect_flux_mcmc_niter={mcmc_niter}_lmsder_niter={lmsder_niter}'):
                        os.makedirs(f'../Photpipe/yse_ztf_fp_VillarFit_noNonDetect_flux_mcmc_niter={mcmc_niter}_lmsder_niter={lmsder_niter}')
                    
                    plt.xlabel('MJD')
                    if (plotType == 'FLUX'):
                        if plt.gca().yaxis_inverted():
                            plt.gca().invert_yaxis()
                        #Flux
                        plt.ylabel('FLUX')
                        plt.errorbar(mjd_pb, flux_pb, fluxerr_pb, alpha=0.7, fmt='o', c=np.unique(plotcolor_pb)[0], label=pb)
                        plt.plot(mjd_grid_pb, model_pb, linestyle='dashed', 
                             linewidth=2.5, alpha=0.9, c=np.unique(plotcolor_pb)[0], label=f'{pb}-fit')
                    else:
                        #Mag
                        plt.ylabel('MAG')
                        if plt.gca().yaxis_inverted():
                            plt.errorbar(mjd_pb, mag_pb, magerr_pb, alpha=0.7, fmt='o', c=np.unique(plotcolor_pb)[0], label=pb)
                        else:
                            plt.gca().invert_yaxis()
                            plt.errorbar(mjd_pb, mag_pb, magerr_pb, alpha=0.7, fmt='o', c=np.unique(plotcolor_pb)[0], label=pb)
                    if (type(redshift) == str):
                        redshift = "N/A"
                        plt.title(f'{objid}, z={redshift}, #residuals > 0.5 mag={totalResidual}')
                    else: 
                        redshift <= 0
                        plt.title(f'{objid}, z={str(round(redshift, 3))}, #residuals > 0.5 mag={totalResidual}')
                    plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=2, fancybox=True, shadow=False)
                if (len(df_pb["Residuals"]) > 0):
                    dfResiduals = df_pb[df_pb["Residuals"] >= thresehold]
                    if (len(dfResiduals) != 0):
                        Rpassband = dfResiduals['PASSBAND']
                        for pb in np.unique(Rpassband):
                            mask = (dfResiduals['PASSBAND'] == pb) & (dfResiduals['PHOTFLAG'] != 0) & (dfResiduals['SNR'] > 4) & (dfResiduals['FLUX'] > 0) #no non-detections
                            Rdf_pb = dfResiduals[mask] 
                            Rdf_pb = Rdf_pb.sort_values(by=['MJD'], ascending=True) # w/ photpipe, some have (near) duplicate MJDs separated
                            Rdf_pb = Rdf_pb.drop_duplicates(subset=['MJD']) # Remove all duplicate MJD rows (prob w/ photpipe output)    

                            Rmjd_pb = np.array(Rdf_pb['MJD'])
                            Rmag_pb = np.array(Rdf_pb['MAG'])
                            Rmagerr_pb = np.array(Rdf_pb['MAGERR'])
                            Rflux_pb = np.array(Rdf_pb['FLUX'])
                            Rfluxerr_pb = np.array(Rdf_pb['FLUXERR'])
                            passband_pb = np.array(Rdf_pb['PASSBAND'])
                            photflag_pb = np.array(Rdf_pb['PHOTFLAG'])
                            plotcolor_pb = Rdf_pb['PLOTCOLOR']

                            if (plotType == "FLUX"):
                                if plt.gca().yaxis_inverted():
                                    plt.gca().invert_yaxis()
                                plt.errorbar(Rmjd_pb, Rflux_pb, Rfluxerr_pb, alpha=0.7, fmt='X', c="yellow")
                            else:
                                if plt.gca().yaxis_inverted():
                                    plt.errorbar(Rmjd_pb, Rmag_pb, Rmagerr_pb, alpha=0.7, fmt='X', c="yellow")
                                else:
                                    plt.gca().invert_yaxis()
                                    plt.errorbar(Rmjd_pb, Rmag_pb, Rmagerr_pb, alpha=0.7, fmt='X', c="yellow")
                        totalResidual = totalResidual + len(dfResiduals)
                if (type(redshift) == str):
                    redshift = "N/A"
                    plt.title(f'{objid}, z={redshift}, #residuals > 0.5 mag={totalResidual}')
                else: 
                    redshift <= 0
                    plt.title(f'{objid}, z={str(round(redshift, 3))}, #residuals > 0.5 mag={totalResidual}')
                plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=2, fancybox=True, shadow=False)
            if (count == len(np.unique(passband))):
                num_of_bands = 0
                no_points_to_plot_mag = 0
                no_points_to_plot_flux = 0
                passband = noFitDf['PASSBAND']
                for pb in np.unique(passband):
                    num_of_bands = num_of_bands + 1
                    mask = (noFitDf['PASSBAND'] == pb) & (noFitDf['PHOTFLAG'] != 0) & (noFitDf['SNR'] > 4) & (noFitDf['FLUX'] > 0) #no non-detections
                    Rdf_pb = noFitDf[mask] 
                    Rdf_pb = Rdf_pb.sort_values(by=['MJD'], ascending=True) # w/ photpipe, some have (near) duplicate MJDs separated
                    Rdf_pb = Rdf_pb.drop_duplicates(subset=['MJD']) # Remove all duplicate MJD rows (prob w/ photpipe output)    
                    Rmjd_pb = np.array(Rdf_pb['MJD'])
                    Rmag_pb = np.array(Rdf_pb['MAG'])
                    Rmagerr_pb = np.array(Rdf_pb['MAGERR'])
                    Rflux_pb = np.array(Rdf_pb['FLUX'])
                    Rfluxerr_pb = np.array(Rdf_pb['FLUXERR'])
                    passband_pb = np.array(Rdf_pb['PASSBAND'])
                    photflag_pb = np.array(Rdf_pb['PHOTFLAG'])
                    plotcolor_pb = Rdf_pb['PLOTCOLOR']
                    
                    #FLux V Mag plots
                    if (plotType == "FLUX"):
                        if plt.gca().yaxis_inverted():
                            plt.gca().invert_yaxis()
                        if (len(noFitDf[mask]) == 0):
                            no_points_to_plot_flux = no_points_to_plot_flux + 1
                            continue
                        plt.errorbar(Rmjd_pb, Rflux_pb, Rfluxerr_pb, alpha=0.7, fmt='o',c=np.unique(plotcolor_pb)[0], label=pb)
                        plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=2, fancybox=True, shadow=False)
                    else:
                        if (len(noFitDf[mask]) == 0):
                            no_points_to_plot_mag = no_points_to_plot_mag + 1
                        else:
                            if plt.gca().yaxis_inverted():
                                plt.errorbar(Rmjd_pb, Rmag_pb, Rmagerr_pb, alpha=0.7, fmt='o', c=np.unique(plotcolor_pb)[0], label=pb)
                                plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=2, fancybox=True, shadow=False)
                            else: 
                                plt.gca().invert_yaxis()
                                plt.errorbar(Rmjd_pb, Rmag_pb, Rmagerr_pb, alpha=0.7, fmt='o', c=np.unique(plotcolor_pb)[0], label=pb)
                                plt.legend(loc='upper right', bbox_to_anchor=(1, 1), ncol=2, fancybox=True, shadow=False)
                #Edge case on redshift type
                if (type(redshift) == str):
                    redshift = "N/A"
                    plt.title(f'{objid}, z={redshift}')
                else: 
                    redshift <= 0
                    plt.title(f'{objid}, z={str(round(redshift, 3))}')

            if ( (num_of_bands == no_points_to_plot_mag) or (num_of_bands == no_points_to_plot_flux) ):
                num_of_bands = 1
                no_points_to_plot_mag = 0
                no_points_to_plot_flux = 0
                continue
            pdf.savefig()  # saves the current figure into a pdf page
            plt.close()

# CSV FUNCTION NOT PLOTTING

In [17]:
#Create df for ZTF g band and ZTF y band and then store it in a csv 
import pandas as pd
def csvFunction(snid_list, meta_list, yse_ztf_fp_df_list):
    mcmc_niter=(4096)
    lmsder_niter=64
    fit = VillarFit('mcmc-lmsder', mcmc_niter=mcmc_niter, lmsder_niter=lmsder_niter) #increase mcmc_niter if fits are not good
    df = pd.DataFrame()   
    mebvExist = 0
    objid = ""
    flux_pb_list, flux_corr_pb_list = [],[]
    #var to check if we are dealing with real or sim since the data we pass is a lil different
    #1 means real
    #0 means sim
    check = 0 
    csvTitle = ""
    if (meta_list[0]['object_id'][0] == '2'):
        print("REAL DATA")
        csvTitle = "Real_Data"
        check = 1
    else:
        type_of_sn = snid_list[0]
        csvTitle =  f'{snid_list[0]}_sims'
  
    # Define column names
    column_names = ['ObjectID', 'X_villar_fit_amplitude', 'X_villar_fit_baseline',
                    'X_villar_fit_peak_time', 'X_villar_fit_rise_time',
                    'X_villar_fit_fall_time', 'X_villar_fit_plateau_slope',
                    'X_villar_fit_plateau_duration', 'X_villar_fit_reduced_chi2',
                    'Y_villar_fit_amplitude', 'Y_villar_fit_baseline',
                    'Y_villar_fit_peak_time', 'Y_villar_fit_rise_time',
                    'Y_villar_fit_fall_time', 'Y_villar_fit_plateau_slope',
                    'Y_villar_fit_plateau_duration', 'Y_villar_fit_reduced_chi2']

    # Initialize dataCSV as an empty DataFrame with NaN values
    dataCSV = pd.DataFrame(columns=column_names)

    # Create a row with NaN values
    nan_row = {col: np.nan for col in column_names}

    row_to_append = pd.DataFrame([nan_row], columns=column_names)

    print('datacsv: ', type(dataCSV))
    dataCSV = pd.concat([dataCSV, pd.DataFrame([nan_row], columns=column_names)], ignore_index=True)
    
    for ix, df in enumerate(yse_ztf_fp_df_list):
        resultList = []
        if(check == 1):
            objid = snid_list[ix]
            # print(objid)
            resultList.append(objid)
        else:
            head = meta_list[ix]['Head']
            uniqueObjId = meta_list[ix]['SNID']
            objid = f'{head}_{uniqueObjId}_{type_of_sn}' #Update if not SN Ia
            resultList.append(objid) 
        # if (check == 1):
            #Checking to see if we are looking at real data type SNIa else skip
            #Update if not SN Ia
            # if (meta_list[ix]['spectype_3class'] != 'SNIa'):
                # continue
        df = df.reset_index(drop=True)
        df['SNR'] = df['FLUX']/df['FLUXERR'] 
        redshift = meta_list[ix]['redshift']
        if (redshift < 0):
            redshift = "N/A"
        peakMJD = df.iloc[df['FLUX'].idxmax()]["MJD"] 
        for i in df['MJD']:
            if ( np.abs(peakMJD - i) > 200):
                df.drop(df.index[df['MJD'] == i], inplace=True)
        if (mebvExist == 1):
            D_id_color = {"X": u"olivedrab",
              "Y": u"darkred",
              "g": u"g", 
              "r": u"r",
              "i": u"m",
              "z": u"k"}
        else:
            D_id_color = {
                  "X ": u"olivedrab",
                  "Y ": u"darkred",
                  "g ": u"g", 
                  "r ": u"r",
                  "i ": u"m",
                  "z ": u"k"}

        df['PLOTCOLOR'] = df.PASSBAND.map(D_id_color)
        results_pb, models_pb = [], []
        totalResidual = 0
        passband = df['PASSBAND']
        count = 0
        countPlot = 0
        noFitDf = df.copy()
        countNum = 0
        if (ix == -1):
            break
        for pb in np.unique(passband):
            mask = (df['PASSBAND'] == pb) & (df['PHOTFLAG'] != 0) & (df['SNR'] > 4) & (df['FLUX'] > 0)#no non-detections

            df_pb = df[mask] 
            df_pb = df_pb.sort_values(by=['MJD'], ascending=True) # w/ photpipe, some have (near) duplicate MJDs separated
            df_pb = df_pb.drop_duplicates(subset=['MJD']) # Remove all duplicate MJD rows (prob w/ photpipe output)    

            mjd_pb = np.array(df_pb['MJD'])
            mag_pb = np.array(df_pb['MAG'])
            magerr_pb = np.array(df_pb['MAGERR'])
            flux_pb = np.array(df_pb['FLUX'])
            fluxerr_pb = np.array(df_pb['FLUXERR'])
            passband_pb = np.array(df_pb['PASSBAND'])
            photflag_pb = np.array(df_pb['PHOTFLAG'])
            plotcolor_pb = df_pb['PLOTCOLOR'] 
            if (len(mjd_pb) < 8):
                count = count + 1
                continue
            else:
                mjd_pb = np.float64(mjd_pb)
                flux_pb = np.float64(flux_pb)
                fluxerr_pb = np.float64(fluxerr_pb)
                result_pb = fit(mjd_pb, flux_pb, fluxerr_pb, sorted=True) #sorted=True now b/c we use sort_value
                results_pb.append(result_pb)

                if (pb == "X " or pb == "X"):
                    if (len(resultList) == 1):
                        for i in range(0, len(result_pb)):
                            resultList.append(result_pb[i])


                if (pb == "Y " or pb == "Y"):
                    for i in range(0, len(result_pb)):
                        resultList.append(result_pb[i])
        if (len(resultList) == 17):
            row_to_append = pd.DataFrame({'ObjectID': [resultList[0]],
                                        'X_villar_fit_amplitude': [resultList[1]],
                                        'X_villar_fit_baseline': [resultList[2]],
                                        'X_villar_fit_peak_time': [resultList[3]],
                                        'X_villar_fit_rise_time': [resultList[4]],
                                        'X_villar_fit_fall_time': [resultList[5]],
                                        'X_villar_fit_plateau_slope': [resultList[6]],
                                        'X_villar_fit_plateau_duration': [resultList[7]],
                                        'X_villar_fit_reduced_chi2': [resultList[8]],
                                        'Y_villar_fit_amplitude': [resultList[9]],
                                        'Y_villar_fit_baseline': [resultList[10]],
                                        'Y_villar_fit_peak_time': [resultList[11]],
                                        'Y_villar_fit_rise_time': [resultList[12]],
                                        'Y_villar_fit_fall_time': [resultList[13]],
                                        'Y_villar_fit_plateau_slope': [resultList[14]],
                                        'Y_villar_fit_plateau_duration': [resultList[15]],
                                        'Y_villar_fit_reduced_chi2': [resultList[16]]})
            dataCSV = pd.concat([dataCSV, row_to_append], ignore_index=True)
    dataCSV = dataCSV.iloc[1:]
    
    print("CSV TITLE: ", csvTitle)
    dataCSV.to_csv(f'{csvTitle}.csv', header=True, index=False)


# Real Data

In [18]:
#THIS IS REAL DATA, format is th same for sim
snid_list, meta_list, yse_ztf_fp_df_list = read_YSE_ZTF_snana_dir(dir_name='spec_yse_dr1') 


# CSV For Real Data

In [19]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined
csvFunction(snid_list, meta_list, yse_ztf_fp_df_list)


REAL DATA
datacsv:  <class 'pandas.core.frame.DataFrame'>


/var/folders/jv/k_v96nss5g52chcq2phrbl240000gn/T/ipykernel_24073/2191943403.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataCSV = pd.concat([dataCSV, row_to_append], ignore_index=True)


CSV TITLE:  Real_Data


In [ ]:
# #print(snid_list)
# for ix, df in enumerate(yse_ztf_fp_df_list):
#     redshift = meta_list[ix]['redshift']
#     print(redshift)
#     print(type(redshift))
#     if (redshift < 0):
#         redshift = "N/A"
#         print("new redshift: ", redshift)
#snid_list[0]
# print(snid_list[0])
# print(meta_list[0])
# print(yse_ztf_fp_df_list[0])


# meta_list[0]['object_id'][0]
# meta_list[0]['num_points']

# Flux for Real Data

In [ ]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined. Change thresehold to your needs
#This is purely for FLUX
plotData(snid_list, meta_list, yse_ztf_fp_df_list, plotType = "FLUX", thresehold = 0.5)

# Mag for Real Data

In [ ]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined. Change thresehold to your needs
#This is purely for MAG
plotData(snid_list, meta_list, yse_ztf_fp_df_list, plotType = "MAG", thresehold = 0.5)

# Sim SNIA Data

In [23]:
#SIMS
#WILL NEED TO CHANGE DIR PATH TO WHEREVER SIMS ARE LOCATED ON YOUR MACHINE
#Update if not SN Ia
snid_list, meta_list, yse_ztf_fp_df_list = read_snana_fits(dir_path='/Users/sammysharief/Downloads/YSE_DR1_SIMS_60k_SNR4_grizXY_SIMLIB_FLUXCALERR_COR_220427/PALEO_YSE_ZTF_MODEL20')

/Users/sammysharief/anaconda3/envs/pgm_project/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sammysharief/anaconda3/envs/pgm_project/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


# CSV for SNIa

In [24]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined
csvFunction(snid_list, meta_list, yse_ztf_fp_df_list)

datacsv:  <class 'pandas.core.frame.DataFrame'>


/var/folders/jv/k_v96nss5g52chcq2phrbl240000gn/T/ipykernel_18651/3318573224.py:146: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataCSV = pd.concat([dataCSV, row_to_append], ignore_index=True)


CSV TITLE:  SNII_sims


# Flux plotting for SNIA

In [ ]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined. Change thresehold to your needs
#This is purely for FLUX
plotData(snid_list, meta_list, yse_ztf_fp_df_list, plotType = "FLUX", thresehold = 0.5)

# Mag plotting for SNIA

In [ ]:
#snid_list, meta_list, yse_ztf_fp_df_list is already defined. Change thresehold to your needs
#This is purely for MAG
plotData(snid_list, meta_list, yse_ztf_fp_df_list, plotType = "MAG", thresehold = 0.5)

In [ ]:
#Storing Old Code
  #Define all the columm names
    # dataCSV = pd.DataFrame(columns = ['ObjectID', 'X_villar_fit_amplitude', 'X_villar_fit_baseline', 
    #                                   'X_villar_fit_peak_time', 'X_villar_fit_rise_time', 
    #                                   'X_villar_fit_fall_time', 'X_villar_fit_plateau_slope', 
    #                                   'X_villar_fit_plateau_duration', 'X_villar_fit_reduced_chi2', 
    #                                   'Y_villar_fit_amplitude', 'Y_villar_fit_baseline', 
    #                                   'Y_villar_fit_peak_time', 'Y_villar_fit_rise_time', 
    #                                   'Y_villar_fit_fall_time', 'Y_villar_fit_plateau_slope', 
    #                                   'Y_villar_fit_plateau_duration', 'Y_villar_fit_reduced_chi2'])
    # #In case Ztf g or Ztf r does not "exist" --> If both don't exist we skip
    # nA = ["N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A"]
    # #Define what var goes to what column
    # dataCSV = dataCSV.append({'ObjectID' : np.nan, 
    #                       'X_villar_fit_amplitude' : np.nan, 
    #                       'X_villar_fit_baseline' : np.nan, 
    #                       'X_villar_fit_peak_time' : np.nan, 
    #                       'X_villar_fit_rise_time' : np.nan, 
    #                       'X_villar_fit_fall_time' : np.nan, 
    #                       'X_villar_fit_plateau_slope' : np.nan, 
    #                       'X_villar_fit_plateau_duration' : np.nan, 
    #                       'X_villar_fit_reduced_chi2' : np.nan,
    #                       'Y_villar_fit_amplitude' : np.nan, 
    #                       'Y_villar_fit_baseline' : np.nan,
    #                       'Y_villar_fit_peak_time' : np.nan, 
    #                       'Y_villar_fit_rise_time' : np.nan,
    #                       'Y_villar_fit_fall_time' : np.nan, 
    #                       'Y_villar_fit_plateau_slope' : np.nan,
    #                       'Y_villar_fit_plateau_duration' : np.nan,
    #                       'Y_villar_fit_reduced_chi2' : np.nan
    #                      }, ignore_index = True)